In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


import numpy as np
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
import requests

#including runtime measurements, accuracy metrics, and model size calculations (Homework 6)
#For training the models with different layers and heads
from itertools import product

import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchinfo import summary
import math
import time
from collections import OrderedDict

#Importing the Swin Transformer model from Hugging Face Transformers library for Problem 3
import transformers
from transformers import SwinForImageClassification, SwinConfig, AutoImageProcessor
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#Check the GPU name and number
'''
devNumber = torch.cuda.current_device()
devName = torch.cuda.get_device_name(devNumber)

print(f"Current device number is: {devNumber}")
print(f"GPU name is: {devName}")'''

Using device: cuda


'\ndevNumber = torch.cuda.current_device()\ndevName = torch.cuda.get_device_name(devNumber)\n\nprint(f"Current device number is: {devNumber}")\nprint(f"GPU name is: {devName}")'

In [ ]:
"""
CIFAR‑100 ─ Vision Transformer scalability study + ResNet‑18 baseline
refactored version (logic preserved, style changed)
"""

# ───────────────────────────── Imports & Globals ──────────────────────────────
import time, random, numpy as np
import torch, torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torchsummary import summary           # <- pip install torchsummary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED          = 42
BATCH         = 64
EPOCHS_VIT    = 20
EPOCHS_RESNET = 10
LR            = 0.001
NUM_CLASSES   = 100

torch.manual_seed(SEED);  np.random.seed(SEED);  random.seed(SEED)

# ────────────────────────────── Data pipeline ─────────────────────────────────
C100_MEAN = (0.5071, 0.4867, 0.4408)
C100_STD  = (0.2675, 0.2565, 0.2761)

transf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(C100_MEAN, C100_STD)
])

train_set = torchvision.datasets.CIFAR100('./data', train=True, download=True,  transform=transf)
test_set  = torchvision.datasets.CIFAR100('./data', train=False,               transform=transf)
train_loader = DataLoader(train_set, batch_size=BATCH, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_set,  batch_size=BATCH, shuffle=False, num_workers=2)

# ───────────────────────────── ViT building blocks ───────────────────────────
class Patchify(nn.Module):
    def __init__(self, img=32, patch=4, ch=3, dim=256):
        super().__init__()
        self.n = (img // patch) ** 2
        self.to_patch = nn.Conv2d(ch, dim, patch, patch)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, dim))
        self.pos = nn.Parameter(torch.zeros(1, self.n + 1, dim))
        nn.init.trunc_normal_(self.pos, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)

    def forward(self, x):
        B = x.size(0)
        x = self.to_patch(x).flatten(2).transpose(1, 2)            # B N D
        cls = self.cls_token.expand(B, -1, -1)
        return torch.cat([cls, x], 1) + self.pos


class MHSA(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        assert dim % heads == 0
        self.h = heads
        self.dk = dim // heads
        self.proj_qkv = nn.Linear(dim, dim * 3)
        self.out = nn.Linear(dim, dim)

    def forward(self, x):
        B, N, D = x.shape
        qkv = self.proj_qkv(x).reshape(B, N, 3, self.h, self.dk).permute(2,0,3,1,4)
        q, k, v = qkv
        att = (q @ k.transpose(-1,-2)) * (self.dk ** -0.5)
        x = (att.softmax(-1) @ v).transpose(1,2).reshape(B, N, D)
        return self.out(x)


class FeedForward(nn.Module):
    def __init__(self, dim, ratio=4, p=0.):
        super().__init__()
        hid = dim * ratio
        self.net = nn.Sequential(
            nn.Linear(dim, hid), nn.GELU(), nn.Dropout(p),
            nn.Linear(hid, dim), nn.Dropout(p)
        )
    def forward(self, x): return self.net(x)


class EncoderBlock(nn.Module):
    def __init__(self, dim, heads, mlp_ratio):
        super().__init__()
        self.norm1, self.att, self.norm2, self.ffn = (
            nn.LayerNorm(dim), MHSA(dim, heads),
            nn.LayerNorm(dim), FeedForward(dim, mlp_ratio)
        )
    def forward(self, x):
        x = x + self.att(self.norm1(x))
        x = x + self.ffn(self.norm2(x))
        return x


class ViT(nn.Module):
    def __init__(self, img=32, patch=4, dim=256, depth=4, heads=4,
                 mlp_ratio=4, classes=100):
        super().__init__()
        self.patch = Patchify(img, patch, 3, dim)
        self.body  = nn.Sequential(*[EncoderBlock(dim, heads, mlp_ratio)
                                     for _ in range(depth)])
        self.norm  = nn.LayerNorm(dim)
        self.head  = nn.Linear(dim, classes)
        self.apply(self._init)

    @staticmethod
    def _init(m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.patch(x)
        x = self.body(x)
        return self.head(self.norm(x[:,0]))

# ────────────────────────────── Train / Evaluate ─────────────────────────────
def loop(model, loader, opt=None):
    train = opt is not None
    model.train() if train else model.eval()
    crit = nn.CrossEntropyLoss()
    hits = tots = 0; t0 = time.time()
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        loss = crit(out, yb)
        if train:
            opt.zero_grad(); loss.backward(); opt.step()
        hits += (out.argmax(1) == yb).sum().item()
        tots += yb.size(0)
    return hits/tots*100, time.time()-t0

# ──────────────────────────── Experiment catalogue ───────────────────────────
variants = [
    dict(tag='ViT‑Tiny',   p=4, d=256, L=4,  H=2, R=2),
    dict(tag='ViT‑Small',  p=8, d=256, L=8,  H=2, R=2),
    dict(tag='ViT‑Medium', p=4, d=512, L=4,  H=4, R=4),
    dict(tag='ViT‑Large',  p=8, d=512, L=8,  H=4, R=4),
]

log = []
for cfg in variants:
    print(f"\n🟢 Training {cfg['tag']}")
    net = ViT(patch=cfg['p'], dim=cfg['d'], depth=cfg['L'],
              heads=cfg['H'], mlp_ratio=cfg['R']).to(device)
    opt = torch.optim.Adam(net.parameters(), lr=LR)
    summary(net, input_size=(3,32,32), batch_size=BATCH, device=str(device))
    epoch_times=[]
    for ep in range(1, EPOCHS_VIT+1):
        _, sec = loop(net, train_loader, opt)
        epoch_times.append(sec)
        print(f"  epoch {ep}/{EPOCHS_VIT} ─ {sec:.2f}s")
    acc,_    = loop(net, test_loader)
    params   = sum(p.numel() for p in net.parameters())/1e6
    flops_ap = sum(p.numel() for p in net.parameters() if p.requires_grad)*2*32*32/1e9
    log.append((cfg['tag'], cfg['p'], cfg['d'], cfg['L'], cfg['H'], cfg['R'],
                params, flops_ap, np.mean(epoch_times), acc))

# ───────────────────────────── ResNet‑18 baseline ────────────────────────────
print("\n🟢 Training ResNet‑18 baseline")
res = torchvision.models.resnet18(num_classes=NUM_CLASSES).to(device)
summary(res, input_size=(3,32,32), batch_size=BATCH, device=str(device))
opt = torch.optim.Adam(res.parameters(), lr=LR)
epoch_times=[]
for ep in range(1, EPOCHS_RESNET+1):
    _, sec = loop(res, train_loader, opt)
    epoch_times.append(sec)
    print(f"  epoch {ep}/{EPOCHS_RESNET} ─ {sec:.2f}s")
acc,_   = loop(res, test_loader)
params  = sum(p.numel() for p in res.parameters())/1e6
flops   = sum(p.numel() for p in res.parameters() if p.requires_grad)*2*32*32/1e9
log.append(('ResNet‑18','N/A','N/A',18,'N/A','N/A',params,flops,np.mean(epoch_times),acc))

# ──────────────────────────────── Final table ────────────────────────────────
print("\n" + "="*118)
hdr = ("Model","Patch","Embed","Depth","Heads","MLP",
       "Params(M)","FLOPs(G)","Time/Epoch(s)","Accuracy")
print("{:<15}{:<8}{:<8}{:<8}{:<8}{:<8}{:<15}{:<15}{:<15}{:<10}".format(*hdr))
print("-"*118)
for r in log:
    print("{:<15}{:<8}{:<8}{:<8}{:<8}{:<8}{:<15.2f}{:<15.2f}{:<15.2f}{:<10.2f}".format(*r))
print("="*118)



🟢 Training ViT‑Tiny
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [64, 256, 8, 8]          12,544
          Patchify-2              [64, 65, 256]               0
         LayerNorm-3              [64, 65, 256]             512
            Linear-4              [64, 65, 768]         197,376
            Linear-5              [64, 65, 256]          65,792
              MHSA-6              [64, 65, 256]               0
         LayerNorm-7              [64, 65, 256]             512
            Linear-8              [64, 65, 512]         131,584
              GELU-9              [64, 65, 512]               0
          Dropout-10              [64, 65, 512]               0
           Linear-11              [64, 65, 256]         131,328
          Dropout-12              [64, 65, 256]               0
      FeedForward-13              [64, 65, 256]               0
     EncoderBlock-

In [ ]:
"""
CIFAR‑100 • Swin‑Tiny, Swin‑Small (frozen) vs. full‑scratch Tiny
The logic matches your last run but the code is rearranged:
  • helper funcs for data, model, train, test
  • dictionary‑based bookkeeping
"""

# ---------- imports ----------
import time
from pathlib import Path
from typing import Dict, Tuple

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import AutoImageProcessor, SwinForImageClassification, SwinConfig
from tqdm.auto import tqdm

# ---------- constants ----------
DEVICE        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE      = 224
BATCH_SIZE    = 32
EPOCHS        = 5
LR            = 2e-5
NUM_CLASSES   = 100

MODELS: Dict[str, str | None] = {
    "tiny":   "microsoft/swin-tiny-patch4-window7-224",
    "small":  "microsoft/swin-small-patch4-window7-224",
    "scratch": None,
}

# ---------- data -------------
proc = AutoImageProcessor.from_pretrained(MODELS["tiny"])
pipeline = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=proc.image_mean, std=proc.image_std),
    ]
)
root = Path("./data")
train_ds = datasets.CIFAR100(root, True,  pipeline, download=True)
test_ds  = datasets.CIFAR100(root, False, pipeline, download=True)

train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_ds,  BATCH_SIZE, shuffle=False, num_workers=2)

# ---------- helpers ----------
def build_swin(tag: str, ckpt: str | None) -> Tuple[nn.Module, list]:
    """return model and list of trainable params"""
    if ckpt is None:             # scratch tiny
        cfg = SwinConfig(
            image_size=IMG_SIZE,
            patch_size=4,
            num_channels=3,
            embed_dim=96,
            depths=[2, 2, 6, 2],
            num_heads=[3, 6, 12, 24],
            window_size=7,
            num_labels=NUM_CLASSES,
        )
        net = SwinForImageClassification(cfg)
        return net.to(DEVICE), list(net.parameters())

    # pretrained path
    net = SwinForImageClassification.from_pretrained(
        ckpt, num_labels=NUM_CLASSES, ignore_mismatched_sizes=True
    )
    for p in net.swin.parameters():      # freeze backbone
        p.requires_grad_(False)
    return net.to(DEVICE), list(net.classifier.parameters())


@torch.no_grad()
def test_top1(model: nn.Module) -> float:
    model.eval()
    good, seen = 0, 0
    for xb, yb in test_loader:
        logits = model(xb.to(DEVICE)).logits
        good += (logits.argmax(1) == yb.to(DEVICE)).sum().item()
        seen += yb.size(0)
    return 100.0 * good / seen

# ---------- main loop ----------
scores = {}
loss_fn = nn.CrossEntropyLoss()

for name, ckpt in MODELS.items():
    print(f"\n⏩ {name}")
    model, trainables = build_swin(name, ckpt)
    opt = torch.optim.Adam(trainables, lr=LR)

    epoch_time = []
    for ep in range(1, EPOCHS + 1):
        model.train()
        t0 = time.perf_counter()
        for xb, yb in tqdm(train_loader, leave=False, desc=f"{name} e{ep}/{EPOCHS}"):
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            loss = loss_fn(model(xb).logits, yb)
            opt.zero_grad(); loss.backward(); opt.step()
        epoch_time.append(time.perf_counter() - t0)
        print(f"  epoch {ep}: {epoch_time[-1]:.2f}s")

    acc = test_top1(model)
    scores[name] = (acc, sum(epoch_time) / EPOCHS)
    print(f"  ✔ accuracy {acc:.2f}% | avg epoch {scores[name][1]:.2f}s")

# ---------- report ----------
print("\n=========== results ===========")
print(f"{'model':<8} {'acc (%)':>8} {'avg‑epoch (s)':>15}")
print("--------------------------------")
for k, (acc, t) in scores.items():
    print(f"{k:<8} {acc:>8.2f} {t:>15.2f}")



⏩ tiny


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([100, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tiny e1/5:   0%|          | 0/1563 [00:00<?, ?it/s]

  epoch 1: 60.07s


tiny e2/5:   0%|          | 0/1563 [00:00<?, ?it/s]

...

  epoch 3: 167.40s


scratch e4/5:   0%|          | 0/1563 [00:00<?, ?it/s]

  epoch 4: 167.40s


scratch e5/5:   0%|          | 0/1563 [00:00<?, ?it/s]

  epoch 5: 167.32s
  ✔ accuracy 36.49% | avg epoch 167.45s

=========== results ===========
model     acc (%)   avg‑epoch (s)
--------------------------------
tiny        66.75           59.98
small       70.56           98.28
scratch     36.49          167.45
